In [50]:
from pulp import *
import pandas as pd

pracownicy = ["Ania","Stefan","Hektor","Olaf","Lidia","Piotr"]
cenah = [100,50,60,40,110,70]
dni = 5
ilosc_pracownikow = [2,1,1,1,3]


prob = LpProblem("Sklep", LpMinimize)
ax = [LpVariable("a{}".format(i+1), cat = "Binary") for i in range(dni)]
sx = [LpVariable("s{}".format(i+1), cat = "Binary") for i in range(dni)]
hx = [LpVariable("h{}".format(i+1), cat = "Binary") for i in range(dni)]
ox = [LpVariable("o{}".format(i+1), cat = "Binary") for i in range(dni)]
lx = [LpVariable("l{}".format(i+1), cat = "Binary") for i in range(dni)]
px = [LpVariable("p{}".format(i+1), cat = "Binary") for i in range(dni)]

lista = [ax,sx,hx,ox,lx,px]
koszt = sum((4*(cena*(sum(x))) for x,cena in zip(lista,cenah)))
prob += koszt

#prob += a1 + s1 + h1 + o1 + l1 + p1 = ilosc_pracownikow[0]
#prob += a2 + s2 + h2 + o2 + l2 + p2 = ilosc_pracownikow[1]
#prob += a3 + s3 + h3 + o3 + l3 + p3 = ilosc_pracownikow[2]
#prob += a4 + s4 + h4 + o4 + l4 + p4 = ilosc_pracownikow[3]
#prob += a5 + s5 + h5 + o5 + l5 + p5 = ilosc_pracownikow[4]






In [51]:
#for number,x in enumerate(lista):
#   print(sum(x[number]))
sum(lista[1])
print(koszt)

1*s1 + 1*s2 + 1*s3 + 1*s4 + 1*s5 + 0